# 검색증강생성(RAG)을 활용한 챗봇 구현

#### 앞 부분은 이전 실습 과정에서 했던 내용과 동일합니다

In [1]:
import boto3
from utils.ssm import parameter_store

region=boto3.Session().region_name
pm = parameter_store(region)

domain_endpoint = pm.get_params(key="opensearch_domain_endpoint", enc=False)
opensearch_domain_endpoint = f"https://{domain_endpoint}"
opensearch_user_id = pm.get_params(key="opensearch_user_id", enc=False)
opensearch_user_password = pm.get_params(key="opensearch_user_password", enc=True)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from botocore.config import Config
from langchain_community.chat_models import BedrockChat
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

region_name = 'us-west-2'
retry_config = Config(
        region_name=region_name,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
boto3_bedrock = boto3.client("bedrock-runtime", region_name=region_name, config=retry_config)

llmchat = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=boto3_bedrock,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"]
    }
)

from langchain.embeddings import BedrockEmbeddings

llmemb = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id="amazon.titan-embed-g1-text-02"
)
dimension = 1536

In [4]:
from utils.rag import qa_chain
from utils.rag import prompt_repo, show_context_used
from utils.rag import retriever_utils, OpenSearchHybridSearchRetriever
from opensearchpy import OpenSearch, RequestsHttpConnection

http_auth = (opensearch_user_id, opensearch_user_password)
os_client = OpenSearch(
        hosts=[
            {'host': opensearch_domain_endpoint.replace("https://", ""),
             'port': 443
            }
        ],
        http_auth=http_auth, 
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection
    )

In [5]:
index_name = "sample_pdf"
opensearch_retriever = OpenSearchHybridSearchRetriever(
    os_client=os_client,
    index_name=index_name,
    llm_text=llmchat, 
    llm_emb=llmemb,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for search
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[1.0, 0.0], # [for semantic, for lexical], Semantic, Lexical search 결과에 대한 최종 반영 비율 정의
    reranker=False, 
    parent_document = False, # enable parent document
    
    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=False,
)

In [6]:
system_prompt = prompt_repo.get_system_prompt()
# 기본 프롬프트 템플릿을 불러와서 활용
print(system_prompt)


                        You are a master answer bot designed to answer user's questions.
                        I'm going to give you contexts which consist of texts, tables and images.
                        Read the contexts carefully, because I'm going to ask you a question about it.
                        


In [8]:
qa = qa_chain(
    llm_text=llmchat,
    retriever=opensearch_retriever,
    system_prompt=system_prompt,
    return_context=True,
    verbose=False
)

In [9]:
query = "특별비용담보 특별약관에서 회사가 보상하는 비용의 범위는?"

response, contexts = qa.invoke(
    query = query
)

print("\n\n\n==============아래는 위 답변에 사용된 컨텍스트입니다==============\n")
show_context_used(contexts)

제2조(비용의 범위) ① 회사가 보상하는 비용의 범위는 아래와 같습니다. 
1. 수색구조비용 
2. 항공운임등 교통비 
3. 숙박비
4. 이송비용
5. 제잡비

특별비용담보 특별약관에서 회사가 보상하는 비용의 범위는 수색구조비용, 교통비, 숙박비, 이송비용, 제잡비(출입국 절차 비용, 교통비, 통신비, 유해처리비 등)입니다.


==============아래는 위 답변에 사용된 컨텍스트입니다==============


-----------------------------------------------
1. Chunk: 539 Characters
-----------------------------------------------
10만원을 한도로 합니다. 제3조(보상하지 아니하는 손해) 회사는 보통약관 제7조(보상하지 아니하는 손해) 제1항 제1호 내지 제3호, 제7호 내지 제12호의 사유로 인하여 생긴
손해는 보상하여 드리지 아니합니다. 제4조(보험금의 지급) 회사는 제2조(비용의 범위)의 비용중 정당하다고 인정된 부분에 대해서 만 보상하여 드리며, 계약자, 피보험자 또는
보험수익자가 타인으로부터 손해배상을 받을 수 있는 경우에는 그 금액을 지급하지 아니합니다. 제5조(보험금의 분담) 제1조(보상하는 손해)의 비용에 대하여 보험금을 지급할 다수의
계약이 체결되어 있는 경우에는 각각의 계약에 대하여 다른 계약이 없는 것으로 하여 산출한 보상책 임액의 합계액이 그 비용을 초과했을 때 회사는 이 계약에 따른 보상책임액의 위의
합계액에 대한 비율에 따라 보험금을 지급하여 드립니다. 제6조(보상한도액) 회사가 이 특별약관에 관하여 지급할 보험금은 보험기간을 통하여 이 특별 약관의 보험가입금액을 한도로
합니다. 제7조(준용규정) 이 특별약관에 정하지 아니한 사항은 보통약관을 따릅니다.
metadata:
 {'source': 'sample1.pdf', 'type': 'sample1', 'timestamp': '2024-04-04T09:10:50.815607', 'id

### RAG 기능을 Streamlit 애플리케이션에 구현

앞서 저장했던 OpenSearch의 벡터 인덱스(`sample_pdf`)에서 질문에 맞는 컨텍스트를 찾아옵니다.

OpenSearch에서 컨텍스트를 찾기 위한 방법으로 **1) 벡터(Semantic) 검색**, **2) 텍스트(Lexical) 검색**을 사용할 수 있습니다.

RAG에서 자연어 검색을 위해 기본적으로 벡터 검색을 사용하지만, 특정 검색어에서는 텍스트 검색에서 더 적합한 컨텍스트를 찾아냅니다.

이를 위해, **벡터 검색과 텍스트 검색을 앙상블**해서 사용하는 **하이브리드 검색**을 최적화 기법으로 사용하기도 합니다. 

아래는 `Hybrid-RAG`를 검색 옵션으로 선택했을 때 **벡터 검색 - 0.51, 텍스트 검색 - 0.49**의 가중치로 컨텍스트를 얻는 기능을 포함합니다.

In [10]:
%%writefile rag.py

import boto3
from botocore.config import Config
from langchain_community.chat_models import BedrockChat
from langchain.callbacks.base import BaseCallbackHandler
from langchain.embeddings import BedrockEmbeddings
from opensearchpy import OpenSearch, RequestsHttpConnection
from .utils.ssm import parameter_store
from .utils.rag import qa_chain, prompt_repo, OpenSearchHybridSearchRetriever

index_name = 'sample_pdf'
region_name = 'us-west-2'
pm = parameter_store(region_name)
opensearch_user_id = pm.get_params(key="opensearch_user_id", enc=False)
opensearch_user_password = pm.get_params(key="opensearch_user_password", enc=True)
domain_endpoint = pm.get_params(key="opensearch_domain_endpoint", enc=False)
opensearch_domain_endpoint = f"https://{domain_endpoint}"

class StreamHandler(BaseCallbackHandler):
    def __init__(self, placeholder):
        super().__init__()
        self.placeholder = placeholder
        self.accumulated_text = ""

    def reset_accumulated_text(self):
        self.accumulated_text = ""
        self.placeholder.text(self.accumulated_text)

    def on_llm_new_token(self, token: str, **kwargs):
        self.accumulated_text += token
        self.placeholder.text(self.accumulated_text)

def initialize_services(chat_box, model_id):
    http_auth = (opensearch_user_id, opensearch_user_password)
    os_client = OpenSearch(
        hosts=[{'host': opensearch_domain_endpoint.replace("https://", ""), 'port': 443}],
        http_auth=http_auth, 
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection
    )

    retry_config = Config(
            region_name=region_name,
            retries={
                "max_attempts": 10,
                "mode": "standard",
            },
        )
    
    boto3_bedrock = boto3.client("bedrock-runtime", region_name=region_name, config=retry_config)
    stream_handler = StreamHandler(chat_box)
    
    llmchat = BedrockChat(
        model_id=model_id,
        client=boto3_bedrock,
        streaming=True,
        callbacks=[stream_handler],
        model_kwargs={
            "max_tokens": 1024,
            "stop_sequences": ["\n\nHuman"]
        }
    )

    llmemb = BedrockEmbeddings(
        client=boto3_bedrock,
        model_id="amazon.titan-embed-g1-text-02"
    )
    dimension = 1536
    print("Bedrock Embeddings Model Loaded")

    return os_client, llmchat, llmemb

def perform_rag_query(query, chat_box, search_type, model_id, ensemble_weights=None):

    os_client, llmchat, llmemb = initialize_services(chat_box, model_id)
    
    if search_type == "Basic-RAG":
        ensemble_weights = [1.0, 0.0]
    elif search_type == "Hybrid-RAG":
        if ensemble_weights is None:
            ensemble_weights = [0.51, 0.49]
    
    opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
        os_client=os_client,
        index_name=index_name,
        llm_text=llmchat, 
        llm_emb=llmemb,
    
        # option for lexical
        minimum_should_match=0,
        filter=[],
    
        # option for search
        fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
        ensemble_weights=ensemble_weights, # [for semantic, for lexical], Semantic, Lexical search 결과에 대한 최종 반영 비율 정의
        reranker=False, 
        parent_document = False, # enable parent document
        
        # option for async search
        async_mode=True,
    
        # option for output
        k=5, # 최종 Document 수 정의
        verbose=False,
    )

    system_prompt = prompt_repo.get_system_prompt()
    qa = qa_chain(
        llm_text=llmchat,
        retriever=opensearch_hybrid_retriever,
        system_prompt=system_prompt,
        return_context=True,
        verbose=False
    )

    response, contexts = qa.invoke(query=query)
    return response, contexts


Writing rag.py


In [11]:
%%writefile ../demo-app.py

import streamlit as st
from RAGchatbot.basic import get_conversation
from RAGchatbot.uploader import upload_and_process_file
from RAGchatbot.rag import perform_rag_query

st.set_page_config(layout="wide")
st.title("Bedrock Q&A Chatbot")

model_options = [
    "anthropic.claude-instant-v1",
    "anthropic.claude-v2:1",
    "anthropic.claude-3-haiku-20240307-v1:0",
    "anthropic.claude-3-sonnet-20240229-v1:0"
]
st.sidebar.title("Model Selection")
selected_model = st.sidebar.selectbox("Type of Cluade Model", model_options, index=3)

st.sidebar.title("Hybrid-RAG Search Balance")
semantic_weight = st.sidebar.slider("Semantic Search Strength", 0.0, 1.0, 0.51, 0.01, help="Adjust the strength of semantic search. The remaining weight is applied to lexical search.")

uploaded_file = st.file_uploader("파일을 업로드하세요", type=["pdf"])
prompt = st.text_input("프롬프트를 입력하세요.")
search_type = st.radio("Search Type", ["Basic", "Basic-RAG", "Hybrid-RAG"])

chat_box = st.empty()

if 'conversation' not in st.session_state or 'stream_handler' not in st.session_state:
    st.session_state.conversation, st.session_state.stream_handler = get_conversation(chat_box, selected_model)

def search_documents(search_type: str, prompt: str):
    if search_type == "Basic":
        st.session_state.stream_handler.reset_accumulated_text()
        st.session_state.conversation.predict(input=prompt)
    elif search_type == "Basic-RAG":
        ensemble_weights = [1.0, 0.0]
        st.session_state.stream_handler.reset_accumulated_text()
        response, contexts = perform_rag_query(prompt, st.session_state.stream_handler.placeholder, search_type, selected_model, ensemble_weights)
        st.write("RAG에 활용된 contexts:")
        for doc in contexts:
            st.write(f"- Source: {doc.metadata['source']}")
            st.write(doc.page_content)
            st.write("---")
    elif search_type == "Hybrid-RAG":
        ensemble_weights = [semantic_weight, 1 - semantic_weight]
        st.session_state.stream_handler.reset_accumulated_text()
        response, contexts = perform_rag_query(prompt, st.session_state.stream_handler.placeholder, search_type, selected_model, ensemble_weights)
        st.write("RAG에 활용된 contexts:")
        for doc in contexts:
            st.write(f"- Source: {doc.metadata['source']}")
            st.write(doc.page_content)
            st.write("---")

if st.button("검색"):
    search_documents(search_type, prompt)

if uploaded_file is not None:
    res = upload_and_process_file(uploaded_file)
    if res:
        st.success("파일이 성공적으로 처리됐습니다.")
    else:
        st.error("파일 처리 중 오류가 발생했습니다.")

Overwriting demo-app.py


<img src="./image/rag-1.png" alt="Image 1" width="800"/>

# 예상 질문 목록
#### 특별비용담보 특별약관에서 회사가 보상하는 비용의 범위는?
#### 한약재 투약비용도 보험금을 지급받을 수 있어?
#### 외교관의 보험가입금액에 대해 알고싶은데, 피보험자가 자녀인 경우, 사망.후유장해에 대한 자녀 1인당 보험가입금액이 얼마야?
#### 계약자가 사망보험을 가입한 후에, 보험금수익자를 변경하려면 어떤 절차를 따라야해?
#### 인질구조비용 특별약관에 가입하면, 인질상태에 놓였을 때 어떤 비용을 보상받을 수 있어?

# 다른 PDF 파일 예시

#### Meta의 생성형 AI 전략에 대해 알려주세요
#### LG 전자는 어떤 생성형 AI 모델을 사용 중인지 알려주세요
#### 10년 후에 생성 AI 시장은 지금보다 얼마나 커질까요?
#### 클라우드에서 엔비디아 GPU의 점유율은 얼마나 될까?
#### 아마존의 생성형 AI 전략이 뭐야?
#### 삼성전자의 생성형 AI 전략이 뭐야?